In [1]:
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import os
import numpy as np

os.chdir('../..')

from src.utils import *
from src.utility import merge_speed_events

In [2]:
speeds_df = pd.read_csv(resources_path('dataset', 'preprocessed', 'speeds_train_imputed_time.csv.gz'))
events_df = pd.read_csv(resources_path('dataset', 'preprocessed', 'events_train.csv.gz'))
sensors_df = pd.read_csv(resources_path('dataset', 'originals', 'sensors.csv.gz'))
speeds_df.shape

(18963685, 9)

In [3]:
speeds_df.dropna(inplace=True)
speeds_df.shape

(11652930, 9)

In [5]:
speeds_events_df = merge_speed_events(speeds_df, events_df).drop_duplicates([KEY, KM, DATETIME])
speeds_events_df.shape

(11652930, 19)

In [6]:
speeds_events_df = pd.merge(speeds_events_df, sensors_df, left_on=[KEY, KM], right_on=[KEY, KM], how='left')
speeds_events_df = speeds_events_df[[KEY, KM, DATETIME, SPEED_AVG, N_CARS, 'IMPUTED', EVENT_TYPE, KM_END, KM_START, 'KM_EVENT', 'step_duration', ROAD_TYPE, LANES, 'EMERGENCY_LANE']]
speeds_events_df.head(10)

,KEY,KM,DATETIME_UTC,SPEED_AVG,N_VEHICLES,IMPUTED,EVENT_TYPE,KM_END,KM_START,KM_EVENT,step_duration,ROAD_TYPE,LANES,EMERGENCY_LANE
0,0.0,333,2018-09-01 00:00:00,231.125907,150.0,False,Pavimentazione,498.0,491.0,496.0,5963.0,1,2,1
1,0.0,333,2018-09-01 00:15:00,225.915947,225.0,False,Pavimentazione,498.0,491.0,496.0,5963.0,1,2,1
2,0.0,333,2018-09-01 00:30:00,228.498770,148.0,False,Pavimentazione,498.0,491.0,496.0,5963.0,1,2,1
3,0.0,333,2018-09-01 00:45:00,231.562904,197.0,False,Pavimentazione,498.0,491.0,496.0,5963.0,1,2,1
4,0.0,333,2018-09-01 01:00:00,229.282707,147.0,False,Pavimentazione,498.0,491.0,496.0,5963.0,1,2,1
5,0.0,333,2018-09-01 01:15:00,230.870995,193.0,False,Pavimentazione,498.0,491.0,496.0,5963.0,1,2,1
6,0.0,333,2018-09-01 01:30:00,229.677286,185.0,False,Pavimentazione,498.0,491.0,496.0,5963.0,1,2,1
7,0.0,333,2018-09-01 01:45:00,239.138228,149.0,False,Pavimentazione,498.0,491.0,496.0,5963.0,1,2,1
8,0.0,333,2018-09-01 02:00:00,231.184505,111.0,False,Pavimentazione,498.0,491.0,496.0,5963.0,1,2,1
9,0.0,333,2018-09-01 02:15:00,237.527496,127.0,False,Pavimentazione,498.0,491.0,496.0,5963.0,1,2,1


In [7]:
idx = speeds_events_df['KM_EVENT'].isnull()
speeds_events_df.loc[idx, 'KM_EVENT'] = speeds_events_df.loc[idx, 'KM_START'] + (speeds_events_df.loc[idx, 'KM_END'] - speeds_events_df.loc[idx, 'KM_START'])/2
speeds_events_df.head(5)

,KEY,KM,DATETIME_UTC,SPEED_AVG,N_VEHICLES,IMPUTED,EVENT_TYPE,KM_END,KM_START,KM_EVENT,step_duration,ROAD_TYPE,LANES,EMERGENCY_LANE
0,0.0,333,2018-09-01 00:00:00,231.125907,150.0,False,Pavimentazione,498.0,491.0,496.0,5963.0,1,2,1
1,0.0,333,2018-09-01 00:15:00,225.915947,225.0,False,Pavimentazione,498.0,491.0,496.0,5963.0,1,2,1
2,0.0,333,2018-09-01 00:30:00,228.498770,148.0,False,Pavimentazione,498.0,491.0,496.0,5963.0,1,2,1
3,0.0,333,2018-09-01 00:45:00,231.562904,197.0,False,Pavimentazione,498.0,491.0,496.0,5963.0,1,2,1
4,0.0,333,2018-09-01 01:00:00,229.282707,147.0,False,Pavimentazione,498.0,491.0,496.0,5963.0,1,2,1


In [8]:
speeds_events_df['DISTANCE'] = speeds_events_df['KM_EVENT'] - speeds_events_df[KM]
speeds_events_df = speeds_events_df[[KEY, KM, DATETIME, SPEED_AVG, N_CARS, 'IMPUTED', EVENT_TYPE, 'DISTANCE', 'step_duration']]
speeds_events_df.head(10)

,KEY,KM,DATETIME_UTC,SPEED_AVG,N_VEHICLES,IMPUTED,EVENT_TYPE,DISTANCE,step_duration
0,0.0,333,2018-09-01 00:00:00,231.125907,150.0,False,Pavimentazione,163.0,5963.0
1,0.0,333,2018-09-01 00:15:00,225.915947,225.0,False,Pavimentazione,163.0,5963.0
2,0.0,333,2018-09-01 00:30:00,228.498770,148.0,False,Pavimentazione,163.0,5963.0
3,0.0,333,2018-09-01 00:45:00,231.562904,197.0,False,Pavimentazione,163.0,5963.0
4,0.0,333,2018-09-01 01:00:00,229.282707,147.0,False,Pavimentazione,163.0,5963.0
5,0.0,333,2018-09-01 01:15:00,230.870995,193.0,False,Pavimentazione,163.0,5963.0
6,0.0,333,2018-09-01 01:30:00,229.677286,185.0,False,Pavimentazione,163.0,5963.0
7,0.0,333,2018-09-01 01:45:00,239.138228,149.0,False,Pavimentazione,163.0,5963.0
8,0.0,333,2018-09-01 02:00:00,231.184505,111.0,False,Pavimentazione,163.0,5963.0
9,0.0,333,2018-09-01 02:15:00,237.527496,127.0,False,Pavimentazione,163.0,5963.0


In [9]:
min_time = pd.to_datetime(speeds_events_df.DATETIME_UTC).astype('int').min()
min_time = (min_time//(10**9))//60
speeds_events_df[DATETIME] = ((pd.to_datetime(speeds_events_df.DATETIME_UTC).astype('int')//(10**9))//60 - min_time)//15
speeds_events_df.head(5)

,KEY,KM,DATETIME_UTC,SPEED_AVG,N_VEHICLES,IMPUTED,EVENT_TYPE,DISTANCE,step_duration
0,0.0,333,0,231.125907,150.0,False,Pavimentazione,163.0,5963.0
1,0.0,333,1,225.915947,225.0,False,Pavimentazione,163.0,5963.0
2,0.0,333,2,228.498770,148.0,False,Pavimentazione,163.0,5963.0
3,0.0,333,3,231.562904,197.0,False,Pavimentazione,163.0,5963.0
4,0.0,333,4,229.282707,147.0,False,Pavimentazione,163.0,5963.0


In [10]:
speeds_events_df[EVENT_TYPE] = speeds_events_df.EVENT_TYPE.fillna('no_event')
speeds_events_df[DISTANCE] = speeds_events_df.DISTANCE.fillna(0)
speeds_events_df['step_duration'] = speeds_events_df.step_duration.fillna(0)
speeds_events_df.sort_values(DATETIME).head()

,KEY,KM,DATETIME_UTC,SPEED_AVG,N_VEHICLES,IMPUTED,EVENT_TYPE,DISTANCE,step_duration
0,0.0,333,0,231.125907,150.0,False,Pavimentazione,163.0,5963.0
8798546,373.0,481,0,231.125907,150.0,True,no_event,0.0,0.0
8790405,373.0,472,0,174.108357,28.0,False,no_event,0.0,0.0
8790402,373.0,445,0,231.125907,150.0,True,no_event,0.0,0.0
561803,14.0,1127,0,246.498783,23.0,False,no_event,0.0,0.0


In [11]:
speeds_events_onehot_df = pd.concat([speeds_events_df, pd.get_dummies(speeds_events_df[EVENT_TYPE])], axis=1)

In [12]:
speeds_events_onehot_df.drop('EVENT_TYPE', axis=1, inplace=True)
speeds_events_onehot_df.head(10)

,KEY,KM,DATETIME_UTC,SPEED_AVG,N_VEHICLES,IMPUTED,DISTANCE,step_duration,Allarme,Barriere,...,Meteo,Opera_arte,Ostacolo_in_carreggiata,Pavimentazione,Regimazione_delle_acque,Segnaletica_orizzontale,Segnaletica_verticale,Veicolo_in_avaria,extended_accident,no_event
0,0.0,333,0,231.125907,150.0,False,163.0,5963.0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0.0,333,1,225.915947,225.0,False,163.0,5963.0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0.0,333,2,228.498770,148.0,False,163.0,5963.0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0.0,333,3,231.562904,197.0,False,163.0,5963.0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0.0,333,4,229.282707,147.0,False,163.0,5963.0,0,0,...,0,0,0,1,0,0,0,0,0,0
5,0.0,333,5,230.870995,193.0,False,163.0,5963.0,0,0,...,0,0,0,1,0,0,0,0,0,0
6,0.0,333,6,229.677286,185.0,False,163.0,5963.0,0,0,...,0,0,0,1,0,0,0,0,0,0
7,0.0,333,7,239.138228,149.0,False,163.0,5963.0,0,0,...,0,0,0,1,0,0,0,0,0,0
8,0.0,333,8,231.184505,111.0,False,163.0,5963.0,0,0,...,0,0,0,1,0,0,0,0,0,0
9,0.0,333,9,237.527496,127.0,False,163.0,5963.0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [13]:
speeds_events_onehot_df.to_csv(resources_path('dataset', 'training', 'train_1.csv.gz'))

## Fitting some models

In [ ]:
import statsmodels.api as sm

In [ ]:
df = pd.read_csv(resources_path('dataset', 'training', 'train_1.csv.gz'))
sensors_df = pd.read_csv(resources_path('dataset', 'originals', 'sensors.csv.gz'))
df.head(5)

In [ ]:
sensors_df.head(5)

In [ ]:
df_1 = df[(df[KEY] == 146.0) & (df[KM] == 1773)]
df_1.sort_values(DATETIME).head(5)